In [4]:
import requests
from bs4 import BeautifulSoup as bs
import cPickle as pk

In [2]:
# from http://stackoverflow.com/questions/16694907/how-to-download-large-file-in-python-with-requests-py
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

def download_reports(reports):
    for y in reports.keys():
        for d in reports[y]:
            fname = download_file(d)

In [3]:
base_url = 'https://www.colorado.gov/pacific/cdphe/medical-marijuana-statistics-and-data'

req = requests.get(base_url)
soup = bs(req.content, 'lxml')
year_stats = soup.findAll('ul')[1].findAll('li')
reports = {}
for y in year_stats:
    new_url = y.findAll('a')[0].get('href')
    year = new_url.split('/')[-1].split('-')[0]
    new_req = requests.get(new_url)
    new_soup =  bs(new_req.content, 'lxml')
    report_links = new_soup.findAll('ul')[1].findAll('a')
    reports[year] = []
    for r in report_links:
        reports[year].append(r.get('href'))

In [6]:
pk.dump(reports, open('reports_dict.pk', 'w'), 2)